# Bibliotecas

In [ ]:
import cohere
import pandas as pd
import os
import dotenv

# Preparing DataFrame

In [ ]:
# reading data
df = pd.read_csv('./dataframes/spotify_millsongdata.csv')
df_aux = pd.read_csv('./dataframes/dataset.csv')

# changing column names
df_aux = df_aux.rename(columns={'track_name':'song', 'artists':'artist'})

In [ ]:
df_aux

In [ ]:
df

In [ ]:
# merging dataframes
df_pop = pd.merge(df, df_aux, on = ['song', 'artist'], how ='inner').drop_duplicates(subset = 'text').reset_index()
df_pop

In [ ]:
# cleaning data
def transform_string(text):
  text = text.replace('\r\n','')
  return text

In [ ]:
lista_aux = []

for val in df_pop['text']:
  val = transform_string(val)
  lista_aux.append(val)

df_pop['lyrics'] = lista_aux

df_pop

In [ ]:
df_pop = df_pop[['artist', 'song', 'lyrics', 'popularity']]
df_pop

# Cohere

In [ ]:
# api client
dotenv.load_dotenv('./.env')

api_key = os.environ["COHERE_API_KEY"]
co = cohere.Client(api_key)

In [ ]:
# text embed calculation
def embed_text(texts):
  output = co.embed(
                model="embed-english-v2.0",
                texts=texts)
  embedding = output.embeddings

  return embedding

In [ ]:
# songs lyrics embeds
df_pop['lyrics_embeds'] = embed_text(df_pop['lyrics'].tolist())

In [ ]:
# dataframe to csv
df_pop.to_csv("./dataframes/data.csv")